## Investigating Molecular Conversions

Our codebase uses 3 different molecular representatins: SMILES strings, RDKit molecules, and NetworkX graphs. The aim of this file is to investigate the loss of information when converting from one molecular representation to another. Specificlly, we investigate the invertibility between SMILES to NetworkX graph and then back to SMILES. 

## Setup

### Change Working Directory

In [1]:
# Change working directory to the parent of the parent of the script

import os

# Get the current working directory
current_directory = os.getcwd()

# Get the parent of the parent directory
parent_parent_directory = os.path.abspath(os.path.join(current_directory, '..', '..'))

# Change the working directory to the parent of the parent directory
os.chdir(parent_parent_directory)

# Verify the change by printing the new working directory
print("New working directory:", os.getcwd())

New working directory: /Users/gordianimperial/Documents/Group Project/bo_molecules


### Imports

In [2]:
# Third-party imports
import pandas as pd
from rdkit import Chem
import networkx as nx
from tdc import Oracle

In [3]:
# Module Imports
from modules.utils.read_sample_store import sample_smiles_from_smiles_csv
from modules.utils.molecular_data_conversion import (
    graph_to_smiles,
    smiles_to_graph,
)

[04/26/24 16:47:46] WARNING  To use the Graphein submodule                                         embeddings.py:44
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             To do so, use the following command: pip install biovec                               
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

[04/26/24 16:47:47] WARNING  To use the Graphein submodule graphein.protein.visualisation, you  visualisation.py:35
                             need to install: pytorch3d                                                            
                             pytorch3d cannot be installed via pip                                                 

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        meshes.py:29
                             install: pytorch3d                                                                    
                             To do so, use the following command: pip install pytorch3d                            

### Specifications

In [4]:
n_samples = 2000

path_to_molecules = "data/zinc.csv"

# List of supported oracles
oracle_names = [
    "albuterol_similarity",
    "amlodipine_mpo",
    "celecoxib_rediscovery",
    "deco_hop",
    "drd2",
    "fexofenadine_mpo",
    "gsk3b",
    "isomers_c7h8n2o2",
    "isomers_c9h10n2o2pf2cl",
    "median1",
    "median2",
    "mestranol_similarity",
    "osimertinib_mpo",
    "perindopril_mpo",
    "QED",
    "ranolazine_mpo",
    "scaffold_hop",
    "sitagliptin_mpo",
    "thiothixene_rediscovery",
    "troglitazone_rediscovery",
    "valsartan_smarts",
    "zaleplon_mpo"
]

### Loading molecular samples

We will randomly sample 2000 molecules from the ZINC database to test molecular conversion.

However, we pass a sample_size of n_samples//2 to the sampling function. This is so because the sampling function doubles the number of samples to safeguard against failed coversions in the downstream tasks.

In [5]:
original_smiles = sample_smiles_from_smiles_csv(path_to_molecules, n_samples//2)

print(f"Sampled {len(original_smiles)} SMILES strings from {path_to_molecules}")

Sampled 2000 SMILES strings from data/zinc.csv


### Utility Functions

To check invertibility of different molecular conversions

In [6]:
def calc_oracle_score_difference(oracle_names: list, original_smiles: list, reconstructed_smiles: list) -> pd.DataFrame:
    """
    Calculate the difference in oracle scores between the original and reconstructed SMILES strings

    Args:
        oracle_names (list): The list of oracle names to use
        original_smiles (list): The list of original SMILES strings
        reconstructed_smiles (list): The list of reconstructed SMILES strings

    Returns:
        pd.DataFrame: The DataFrame containing the oracle score differences
    """
    results = []

    # Iteratively calcualte difference for all oracles
    for oracle_name in oracle_names:
        oracle = Oracle(name=oracle_name)
        # Iterate over all original and reconstructed SMILES pairs
        for orig_smiles, recon_smiles in zip(original_smiles, reconstructed_smiles):
            orig_score = oracle(orig_smiles)
            recon_score = oracle(recon_smiles)
            diff = orig_score - recon_score
            
            row = [oracle_name, orig_smiles, recon_smiles, orig_score, recon_score, diff]
            results.append(row)

    results = pd.DataFrame(results, columns=['oracle', 'original_smiles', 'reconstructed_smiles', 'original_score', 'reconstructed_score', 'score_difference'])

    return results

## Experiments

### Test SMILES -> NetworkX Graph -> SMILES Conversion

In [7]:
# Convert SMILES to NetworkX graphs
graphs = [smiles_to_graph(smiles) for smiles in original_smiles]

# Convert NetworkX graphs to SMILES
reconstructed_smiles = [graph_to_smiles(graph) for graph in graphs]

# Indices for which the original and reconstructed SMILES strings differ
error_idx = [i for i, (s1, s2) in enumerate(zip(original_smiles, reconstructed_smiles)) if s1 != s2]

# Report errors
print(f"Failed to exactly reconstruct {len(error_idx)} / {len(original_smiles)} SMILES.")
print("----------------------")

for i in error_idx:
    print(f"Original: {original_smiles[i]}")
    print(f"Reconstructed: {reconstructed_smiles[i]}")
    print("----------------------")

Failed to exactly reconstruct 34 / 2000 SMILES.
----------------------
Original: COC(=O)c1cc(C[NH+]2CCC(C(=O)N3C[C@H](C)O[C@H](C)C3)CC2)c[nH]1
Reconstructed: COC(=O)c1cc(C[NH+]2CCC(C(=O)N3C[C@@H](C)O[C@@H](C)C3)CC2)c[nH]1
----------------------
Original: Cc1ccc(C(=O)N[C@@H]2N=C3[C@H](CC[C@@H]3C(=O)NCCc3c[nH]c4ccccc34)S2)cc1
Reconstructed: Cc1ccc(C(=O)N[C@@H]2N=C3[C@@H](C(=O)NCCc4c[nH]c5ccccc45)CC[C@@H]3S2)cc1
----------------------
Original: CCC[NH+](CCC)[C@H]1[C@@H](C(=O)[O-])[C@H]2CC[C@@H]1C2
Reconstructed: CCC[NH+](CCC)[C@@H]1[C@@H]2CC[C@@H](C2)[C@@H]1C(=O)[O-]
----------------------
Original: C[C@@H]1SCCC[C@H]1[NH2+]CC[C@@H](C)c1ccccc1
Reconstructed: C[C@H](CC[NH2+][C@@H]1CCCS[C@H]1C)c1ccccc1
----------------------
Original: C#CCOc1ccc(F)cc1NC(=O)C(=O)N[C@@H]1C[C@H]2C[C@@H]1[C@H]1CCC[C@@H]12
Reconstructed: C#CCOc1ccc(F)cc1NC(=O)C(=O)N[C@@H]1C[C@H]2C[C@@H]1[C@H]1CCC[C@H]21
----------------------
Original: C[C@H]1CC[C@@H](C)CN1C(=O)c1ccc(S(N)(=O)=O)o1
Reconstructed: C[C@@H]1CC[C@H](C

#### Understanding Failure Modes

##### Stereochemistry

Since we are working with 2D molecular graphs, our conversions loose information regarding exact 3D orientation and thus, sometimes incorrectly converts one isomer into another. 

In the SMILES representation, '@' and '@@' represents the clockwise (*D*-isomer) and counter-clockwise (*L*-isomer) orientation respectively.

We observed that all of the above error arise because of this issue. Specifically, '@' gets converted to '@@' and vice versa. This also leads to changes in the indexing of atoms in a ring structure and thus, changes the ordering of atoms in the reconstructed SMILES.

#### Analysing Difference in Oracle Scores

Next, we analyse the impact of our inexact invertibility on the oracle scores. 

In [8]:
# Calculate difference between Oracle scores for all sampled SMILES
oracle_diff_df = calc_oracle_score_difference(oracle_names, original_smiles, reconstructed_smiles)

Found local copy...
100%|██████████| 27.8M/27.8M [00:10<00:00, 2.66MiB/s]
Done!


Analyse SMILES for which oracle scores before and after reconstruction are not identical

In [9]:
# Filter inexact scores
inexact_scores_df = oracle_diff_df[oracle_diff_df['score_difference']!=0.0]

# Report SMILES with inexact match
inexact_score_smiles = inexact_scores_df['original_smiles'].unique()
inexact_score_oracles = inexact_scores_df['oracle'].unique()

print(f"Oracle scores didn't match exactly for {len(inexact_score_smiles)} / {len(original_smiles)} SMILES.")
print("----------------------")
print(f"Difference observed for below Oracles:\n {inexact_score_smiles}")

Oracle scores didn't match exactly for 11 / 2000 SMILES.
----------------------
Difference observed for below Oracles:
 ['C[C@H]1C[C@@H](C)CCN(C(=O)N[C@@H](c2ccc(Cl)cc2)c2ncon2)C1'
 'C[C@@]12CC/C(=N\\OCC(=O)[O-])C=C1CC[C@@H]1[C@@H]2CC[C@]2(C)[C@@H]1CC[C@]2(C)O'
 'C[C@@H](CNC(=O)N1CCCCC[C@@H]1c1ccco1)N1CCOC[C@H]1C'
 'CC1(C)[C@@H](C(=O)Oc2cccc3cccnc23)[C@@H]1C=C(Br)Br'
 'COc1ccc(C(=O)N2C[C@H](c3cccc(F)c3)[C@@H](C[NH+]3CCCC3)C2)cc1OC'
 'C[C@H]1CN(Cc2cc(F)cc(C#CC[NH3+])c2)[C@H](C)CO1'
 'C[C@H](NC(=O)N[C@H](C)Cn1cc[nH+]c1)C(=O)N1CCCCC1'
 'C[C@H]1CN(C(=O)[C@@H](C)n2cccn2)c2ccccc2S1'
 'C[C@H]1CC(=O)[C@H]2[C@@]3(O)C(=O)c4cccc(O)c4[C@@H]4O[C@@]43[C@@H](O)C[C@]2(O)C1'
 'CC(C)(C)c1cc(C[NH+]2CCC3(CC2)[C@@H]([O-])C[C@H]3O)n[nH]1'
 'O=C(Nc1ccc(OC(F)F)c(Cl)c1)[C@H]1[C@@H]2C=C[C@@H](O2)[C@@H]1C(=O)[O-]']


Next, we analyse the observed differences within some error tolerance. Please note that the oracle scores lie between 0 and 1. Therefore, a tolerance of 0.001 seems sufficient. 

In [10]:
# Define an error tolerance
error_tolerance = 0.001

bounded_error_df = oracle_diff_df[oracle_diff_df['score_difference'].abs() > error_tolerance]

# Report SMILES with error greater than our tolerance
bounded_error_smiles = bounded_error_df['original_smiles'].unique()
bounded_error_oracles = bounded_error_df['oracle'].unique()

print(f"Oracle scores didn't match exactly for {len(bounded_error_smiles)} / {len(original_smiles)} SMILES.")
print("----------------------")
print(f"Difference observed for below Oracles:\n {bounded_error_oracles}")

Oracle scores didn't match exactly for 0 / 2000 SMILES.
----------------------
Difference observed for below Oracles:
 []
